In [3]:
# create sql connection
from sqlalchemy import create_engine
import pandas as pd
import json
import urllib.parse
import psycopg2
import os 
with open("credentials.json") as file:
    login = json.load(file)
    
username = login['user']
password = urllib.parse.quote(login['password'])
host = login['host']
port = login['port']

engine = create_engine("postgresql://{}:{}@{}:{}/postgres".format(username,password,host,port))

In [31]:
# merge data

with engine.connect() as cnn:
    """
    With any proxy parameter == TRUE, this questionnair is answered by others, so we will drop them all
    """
    
    df = pd.read_sql("""
                            SELECT 
                                a."UID",
                                a."householdUID", 
                                a."communityUID",
                                proxy_2,
                                ba001,
                                ba009,
                                ba011,
                                ba014,
                                ba015,
                                ba016,
                                ba017,
                                proxy,
                                proxy_5,
                                da001,
                                da002_1_,
                                da002_2_,
                                da002_3_,
                                da002_4_,
                                da002_5_,
                                da002_6_,
                                da002_7_,
                                da002_8_,
                                da002_9_,
                                da002_10_,
                                da002_11_,
                                da002_12_,
                                da002_13_,
                                da002_14_,
                                da013,
                                da019,
                                da022,
                                da025,
                                da027,
                                da029,
                                da030,
                                da031,
                                da038_s1,
                                da038_s2,
                                da038_s3,
                                da038_s4,
                                da038_s5,
                                da038_s6,
                                da038_s7,
                                da038_s8,
                                da038_s9,
                                da040,
                                da049_1,
                                da050_1,
                                da051,
                                db001,
                                db003,
                                db005,
                                db007,
                                db009,
                                db011,
                                db012,
                                db014,
                                db016,
                                db018,
                                db020,
                                db022,
                                fa001,
                                fc006,
                                fc008
                            from charles.work_retirement a
                            join charles.health_status_and_functioning using("UID")
                            join charles.demographic_background using("UID")
                                     """, con = cnn)
    
df.to_csv('df.csv')

In [53]:
# drop survey done by others
df = pd.read_csv('df.csv', index_col=0)
proxy_var = ['proxy','proxy_2','proxy_5']
mask = df[proxy_var].apply(lambda x: x.str.contains('1 Yes')).any(axis=1)
df = df[~mask].drop(labels=proxy_var, axis=1)

* Now based on data types, will will process different proprecessing for them:
    * imputer
    * binary: keep it with 0 or 1
    * categorical:
      * if with meaning, use ordinal encode, positive impact with larger number
      * if not, use one-hot encode
    * numerical:
      * scalar

In [54]:
unique_values = df.apply(lambda x: len(x.unique()))
pd.set_option('display.max_rows', 500)
unique_values


UID             17563
householdUID    10784
communityUID      446
ba001               2
ba009               4
ba011               6
ba014               7
ba015               7
ba016               3
ba017               7
da001               6
da002_1_            5
da002_2_            5
da002_3_            5
da002_4_            5
da002_5_            5
da002_6_            5
da002_7_            5
da002_8_            5
da002_9_            5
da002_10_           5
da002_11_           5
da002_12_           4
da002_13_           5
da002_14_           5
da013               3
da019               3
da022               3
da025               3
da027               6
da029               7
da030              41
da031              71
da038_s1            3
da038_s2            3
da038_s3            3
da038_s4            3
da038_s5            3
da038_s6            3
da038_s7            3
da038_s8            3
da038_s9            3
da040               3
da049_1            72
da050_1            42
da051     

array(['2 No', '1 Yes', 0], dtype=object)